In [86]:
import sys
# sys.path.append("/Users/bubble/Desktop/Project/Infrasound Sensor/Layout/Code/acousticsensor")
import numpy as np
import gdsfactory as gf

cell_temp = gf.Component()

In [87]:
# pad and lines
pad = gf.Component()

# lines
line_base = gf.Component()
line_half = gf.Component()
# define the cross section
w1 = 10
w2 = 0.4  
s1 = gf.Section(width=w1, offset=0, layer=(2, 0), name="1", port_names=("o1", "o2"))
X1 = gf.CrossSection(sections=[s1])
s2 = gf.Section(width=w2, offset=0, layer=(2, 0), name="1", port_names=("o1", "o2"))
X2 = gf.CrossSection(sections=[s2])
# define the path
# define distance
d1 = 200
radius = 20
h1 = 20
d5 = 0.4
p1 = gf.path.straight(length=d1)
p = p1
wg1 = gf.path.extrude(p, X1)
p4 = gf.path.arc(radius=radius, angle=-90)
Xtrans = gf.path.transition(cross_section1=X1, cross_section2=X2, width_type="sine")
curve_transition = gf.path.extrude_transition(p4, Xtrans)

wg1_ref = line_base << wg1
wg2_ref = line_base << curve_transition
wg2_ref.connect("o1", wg1_ref.ports["o2"])

P = gf.Path()
p5 = gf.path.straight(length=h1)
p6 = gf.path.straight(length=d5)
P += p5
P.end_angle += 90  # "Turn" 90 deg (left)
P += p6
wg3 = gf.path.extrude(P, X2)
wg3_ref = line_base << wg3
wg3_ref.connect("o1", wg2_ref.ports["o2"])

# line_base.show()
d = d1+radius+d5
line_base_ref1 = line_half << line_base
line_base_ref2 = line_half << line_base
line_base_ref2.dmirror_x(d)
pad.add_ref(line_half)
pad.add_ref(line_half).drotate(angle=180, center=(2*d, 0))
# pad.show()




Unnamed_2401: ports [], vinsts=[] info=Info() kcl=KCLayout(name='DEFAULT', layout=<klayout.dbcore.Layout object at 0x1104013d0>, layer_enclosures=LayerEnclosureModel(root={'5c226817': LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure'), '4fb2b992': LayerEnclosure(layer_sections={}, main_layer=9/0, yaml_tag='!Enclosure'), '56396fc8': LayerEnclosure(layer_sections={}, main_layer=SLAB90 (3/0), yaml_tag='!Enclosure'), 'a4b5fcfc': LayerEnclosure(layer_sections={}, main_layer=8/0, yaml_tag='!Enclosure'), '9e36fa27': LayerEnclosure(layer_sections={}, main_layer=DEEPTRENCH (4/0), yaml_tag='!Enclosure'), '6b4bcfc2': LayerEnclosure(layer_sections={}, main_layer=N (20/0), yaml_tag='!Enclosure'), 'a5f0d270': LayerEnclosure(layer_sections={}, main_layer=P (21/0), yaml_tag='!Enclosure')}), cross_sections={'5c226817_10000': SymmetricalCrossSection(width=10000, enclosure=LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure'), name='5c226817_10

In [88]:
# pads
pad_size = 500
h2 = 100
w3 = 20
line_vertical = gf.components.rectangle(size=(w3, h2), layer=(2, 0))
pad.add_ref(line_vertical).move((2*d-w3/2, -h2-w1/2))
sg_pad = gf.components.rectangle(size=(pad_size, pad_size), layer=(2, 0))
pad.add_ref(sg_pad).move((-pad_size, w1/2-pad_size))
pad.add_ref(sg_pad).move((4*d, w1/2-pad_size))
pad.add_ref(sg_pad).move((2*d-pad_size/2, -w1/2-pad_size-h2))

# pad.show()



Unnamed_2401: ports ['e1', 'e2', 'e3', 'e4'], vinsts=[] info=Info() kcl=KCLayout(name='DEFAULT', layout=<klayout.dbcore.Layout object at 0x1104013d0>, layer_enclosures=LayerEnclosureModel(root={'5c226817': LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure'), '4fb2b992': LayerEnclosure(layer_sections={}, main_layer=9/0, yaml_tag='!Enclosure'), '56396fc8': LayerEnclosure(layer_sections={}, main_layer=SLAB90 (3/0), yaml_tag='!Enclosure'), 'a4b5fcfc': LayerEnclosure(layer_sections={}, main_layer=8/0, yaml_tag='!Enclosure'), '9e36fa27': LayerEnclosure(layer_sections={}, main_layer=DEEPTRENCH (4/0), yaml_tag='!Enclosure'), '6b4bcfc2': LayerEnclosure(layer_sections={}, main_layer=N (20/0), yaml_tag='!Enclosure'), 'a5f0d270': LayerEnclosure(layer_sections={}, main_layer=P (21/0), yaml_tag='!Enclosure')}), cross_sections={'5c226817_10000': SymmetricalCrossSection(width=10000, enclosure=LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosur

In [89]:
# cell
L = [250, 500, 750, 1000]
g = 30
column_list = [1, 2, 4, 8, 16]
k = len(L) * len(column_list)
w_total = 32

mesh = [None] * k
for i in range(k):
    mesh[i] = gf.Component()
block = gf.Component()
block_temp = gf.Component()
origin = [0, 0]
count = 0

for i in range(len(L)):
    for j in range(len(column_list)):
        w = w_total / column_list[j]
        # single grid
        column = column_list[j]
        height = L[i] - w
        for m in range(column-1):
            sg = gf.components.rectangle(size=(g, height), layer=(8, 0))
            sf_ref = mesh[count] << sg
            sf_ref.move((origin[0]+m*(g+w), 0))    # hole between the beams
        fra_width = (column-1) * (g + w) + w
        beam_frame = gf.components.rectangle(size=(fra_width, L[i]), layer=(9, 0))
        beam_ref = mesh[count] << beam_frame
        beam_ref.movex(origin[0]-w)
        
        # length mark
        T = gf.components.text(f"{count}-L={L[i]}, n={column_list[j]}", size=50, layer=(1, 0))
        T_ref = block << T
        # add pad
        pad_ref = block << pad
        # add backside frame/etch
        size = 1000
        back_temp = gf.Component()
        frame = gf.components.rectangle(size=(size, L[i]+250), layer=(4, 0))
        backside = gf.components.rectangle(size=(size+371.72, L[i]+250+371.72), layer=(3, 0))
        back_temp << frame
        back_temp.add_ref(backside).movex(-371.72/2).movey(-371.72/2)
        mesh[count].add_ref(back_temp).movex((-size+fra_width)/2-w)        
        mesh_ref = block_temp << mesh[count]
        if j == 0:
            mesh_ref.movex(w/2)
        else:
            mesh_ref.movex(w/2-((column_list[j]//2)-1)*(w+g))
        # adjust position
        spacing = 2700
        mesh_ref.movey(2250*i-250).movex(spacing*j)
        T_ref.movey(2250*i-250-700).movex(spacing*j)
        pad_ref.move((-3*d, 2250*i-250-radius)).movex(spacing*j)
        count += 1
beam_temp = gf.boolean(A = block_temp, B = block_temp,  operation="not", layer1=(4, 0), layer2=(9, 0), layer=(1, 0))
block << beam_temp
beam_temp = gf.boolean(A = block_temp, B = block_temp,  operation="or", layer1=(8, 0), layer2=(15, 0), layer=(1, 0))
block << beam_temp
beam_temp = gf.boolean(A = block_temp, B = block_temp,  operation="or", layer1=(3, 0), layer2=(15, 0), layer=(3, 0))
block << beam_temp
# block.show()


Unnamed_2427: ports [], vinsts=[] info=Info() kcl=KCLayout(name='DEFAULT', layout=<klayout.dbcore.Layout object at 0x1104013d0>, layer_enclosures=LayerEnclosureModel(root={'5c226817': LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure'), '4fb2b992': LayerEnclosure(layer_sections={}, main_layer=9/0, yaml_tag='!Enclosure'), '56396fc8': LayerEnclosure(layer_sections={}, main_layer=SLAB90 (3/0), yaml_tag='!Enclosure'), 'a4b5fcfc': LayerEnclosure(layer_sections={}, main_layer=8/0, yaml_tag='!Enclosure'), '9e36fa27': LayerEnclosure(layer_sections={}, main_layer=DEEPTRENCH (4/0), yaml_tag='!Enclosure'), '6b4bcfc2': LayerEnclosure(layer_sections={}, main_layer=N (20/0), yaml_tag='!Enclosure'), 'a5f0d270': LayerEnclosure(layer_sections={}, main_layer=P (21/0), yaml_tag='!Enclosure')}), cross_sections={'5c226817_10000': SymmetricalCrossSection(width=10000, enclosure=LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure'), name='5c226817_10

In [ ]:
# boolean operation
cell_temp.add_ref(block).move((2000, 1800))
# frame
frame1 = gf.components.rectangle(size=(15000, 10830), layer=(20, 0))
cell_temp << frame1
frame2 = gf.components.rectangle(size=(20000, 15830), layer=(21, 0))
frame2_ref = cell_temp << frame2
frame2_ref.move((-2500, -2500))

cell_mesh_resistive = gf.Component()
cell_ref = cell_mesh_resistive << cell_temp
cell_ref.move((2500, -7500))
cell_mesh_resistive.show()
# cell_mesh_resistive.write_gds("mesh.gds")
# cell_mesh_resistive.plot()